## Lab2 目標：抓取大安區公所的逐三小時預報
### 有些欄位有 colspan = ? 必須額外展開才能塞進 dataframe

In [69]:
import sys
import requests
from bs4 import BeautifulSoup

In [70]:
url = 'http://www.cwb.gov.tw/V7/forecast/town368/3Hr/6300300.htm'
req = requests.get(url)
req.encoding = 'utf-8' # 不指定會發生編碼錯誤

### 如果要用類似 scrapy shell debug 的功能，uncomment 下方的 code block

In [71]:
#from scrapy.http import TextResponse
#response = TextResponse(req.url, body=req.text, encoding='utf-8') # 變數特別命名的跟 scrapy shell 開起來一樣
#soup = BeautifulSoup(response.text, 'html.parser')

### 使用 requests 得到的 html 內容，用 bs4 parse 所需資訊

In [72]:
soup = BeautifulSoup(req.text, 'html.parser')
trs = soup.find_all('tr')

In [73]:
import pandas as pd
columns = ['record_t', # 0
           'weekday', # 1
           'wx', # 2
           't', # 3 
           'at', # 4
           'beaufort', # 5
           'wind_dir', # 6
           'rh', # 7
           'pop', #8 
           'ci'] # 9
df = pd.DataFrame(columns=columns)

## 以下的 len() == 3 ，先整理 48 小時內的 timestamp

In [74]:
# day repeat time store in colspans
# date store in dates
# days store in days
# len(colspans) == len(dates) == len(days) == 2 or 3
import re
import datetime
year_s = []
year_s.append("%d" % datetime.datetime.now().year)
year_s.append("%d" % (datetime.datetime.now() + datetime.timedelta(days=1)).year) # in case tmrr is next year
year_s.append("%d" % (datetime.datetime.now() + datetime.timedelta(days=2)).year) # in case it's next year in 2 days
colspans = []
dates = []
days = []
k = 0
for idx, td in enumerate(trs[0].findAll('td')): # trs[0] 是時間相關的列
    if idx > 0:
        if td.has_attr('colspan'):
            colspans.append(td.attrs['colspan'])
        else:
            colspans.append("1")
        days.append(re.findall('[一|二|三|四|五|六|日]', td.text)[0]) # 用正規表示式把星期"幾"選出來
        month_n_date = re.findall('\d+', td.text)
        dates.append(year_s[k] + '-' + month_n_date[0] + '-' + month_n_date[1])
        k+=1

## 以下的 len() == 17，行數=17，列數不含 index = 10
### 範例

| pk | record_t(0) | weekday(1) | wx(2) | t(3) | at(4) | beaufort(5) | wind_dir(6) | rh(7) | pop(8) | ci(9) | 
| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| 1 | 2017-08-15 09:00 | 星期二 | 晴天 | 33 | 38 | 2 | 偏西風 | 78% | 0% | 悶熱

### 處理第 0:1 行

In [75]:
record_ts = []
weekdays = []
hours = trs[1].findAll('span')
k = 0
for i in range(0, len(colspans)):
    for j in range(0, int(colspans[i])):
        record_ts.append(dates[i] + ' ' + hours[k].text)
        k+=1
        weekdays.append(days[i])

df['record_t'] = record_ts
df['weekday'] = weekdays

### 處理第 2 行

In [76]:
wxs = []
for img in trs[2].findAll('img'):
    wxs.append(img.attrs['alt'])

df['wx'] = wxs

### 處理第 3:9 行，除了第 8 行

In [77]:
vals = []
for i in range(3, 10):
    if i is not 8: # 降雨機率可能要展開 colspan == 2
        tds = trs[i].findAll('td')
        for idx, td in enumerate(tds):
            if idx > 0:
                vals.append(td.text)
        df.iloc[:,i] = vals
        vals = []

### 處理第 8 行

In [78]:
pops = [] # probability of precipitation
rep = 0
for idx, td in enumerate(trs[8].findAll('td')):
    if idx > 0:
        if td.has_attr('colspan'):
            rep = int(td.attrs['colspan'])
        else:
            rep = 1
        for i in range(0, rep):
            pops.append(td.text)

df['pop'] = pops

### 呈現

In [79]:
import copy
dfc = copy.copy(df)
dfc.index += 1 # in case we need to import to mysql
dfc

,record_t,weekday,wx,t,at,beaufort,wind_dir,rh,pop,ci
1,2017-08-22 21:00,二,短暫陣雨或雷雨,31,29,5,東南風,61%,30%,悶熱
2,2017-08-23 00:00,三,多雲,30,31,3,東南風,65%,10%,悶熱
3,2017-08-23 03:00,三,多雲,29,33,<=1,東南風,69%,10%,舒適
4,2017-08-23 06:00,三,多雲,29,31,2,東南風,66%,30%,舒適
5,2017-08-23 09:00,三,短暫陣雨,32,35,2,偏南風,56%,30%,悶熱
6,2017-08-23 12:00,三,午後短暫雷陣雨,35,39,2,西南風,53%,30%,悶熱
7,2017-08-23 15:00,三,多雲,34,39,2,西南風,71%,30%,悶熱
8,2017-08-23 18:00,三,短暫陣雨,32,37,<=1,偏南風,76%,30%,悶熱
9,2017-08-23 21:00,三,短暫陣雨,31,34,2,偏東風,72%,30%,悶熱
10,2017-08-24 00:00,四,多雲,30,33,<=1,偏東風,67%,10%,悶熱


### 存檔

In [80]:
out_csv = 'daan_3hr.csv'
dfc.to_csv(out_csv, sep=',', encoding='utf-8')